# Digit Recognition with Simple CNN

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Reading and Preparing data

In [ ]:
train_data = pd.read_csv("../input/digit-recognizer/train.csv")
test_data = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.head(10)

In [ ]:
X = train_data.drop(["label"],axis = 1).values
Y = train_data["label"].values
plt.figure(figsize = (12,6))
sns.countplot(Y)

In [ ]:
plt.imshow(X[0].reshape([28,28]))

In [ ]:
plt.imshow(X[1].reshape([28,28]))

In [ ]:
plt.imshow(X[7].reshape([28,28]))

In [ ]:
plt.imshow(X[14].reshape([28,28]))

In [ ]:
X = X.reshape([42000,28,28,1])
Y = Y.reshape([42000,1])

In [ ]:
# convert to one-hot-encoding
from keras.utils.np_utils import to_categorical 
Y = to_categorical(Y, num_classes = 10)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 14)


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
x_train = x_train/255
x_test = x_test/255

# Defining the CNN architecture

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

## Compiling the model (Optimizer=Adam)

In [ ]:
from keras.optimizers import Adam
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

## Learning Rate Reduction

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.6, 
                                            min_lr=0.00001)

In [ ]:
batch_size = 64
epochs = 30

## Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator( 
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,
        shear_range = 0.1,
        horizontal_flip=False,  
        vertical_flip=False
        )
train_datagen.fit(x_train)


## Fitting the model

In [ ]:
history = model.fit(
            train_datagen.flow(x_train,y_train,batch_size = batch_size),
            validation_data = (x_test,y_test),
            batch_size = batch_size,
            steps_per_epoch = x_train.shape[0]//batch_size,
            epochs = epochs,
            verbose = 1,
            callbacks=[learning_rate_reduction]
            )

## Evaluation and Accuracy,Loss plots

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(acc,color = 'green',label = 'Training Acuracy')
plt.plot(val_acc,color = 'red',label = 'Validation Accuracy')
plt.legend()

In [ ]:
plt.figure()
plt.plot(loss,color = 'green',label = 'Training Loss')
plt.plot(val_loss,color = 'red',label = 'Validation Loss')
plt.legend()

In [ ]:
data = test_data.values
data = data.reshape([28000,28,28,1])
print(data.shape)
data = data/255

In [ ]:
test_pred = model.predict(data)
test_pred = np.argmax(test_pred,axis=1)
print(test_pred.shape)

In [ ]:
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
sample_submission

In [ ]:
index = sample_submission.ImageId
data = {'ImageId' : index,'Label': test_pred}
df = pd.DataFrame(data)
df.head

In [ ]:
df.to_csv('submission2.csv', index=False)